# One hot

In [1]:
from keras.datasets import reuters
import numpy as np 
np_load_old = np.load
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

Using TensorFlow backend.


In [2]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results


# 훈련 데이터를 벡터로 변환합니다
x_train = vectorize_sequences(train_data)
# 테스트 데이터를 벡터로 변환합니다
x_test = vectorize_sequences(test_data)

In [3]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [3]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [4]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [7]:
history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=10,
                   batch_size=512,
                   validation_data=(x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 7982 samples, validate on 1000 samples
Epoch 1/5
7982/7982 [==============================] - 2s 199us/step - loss: 3.2171 - acc: 0.4230 - val_loss: 2.4325 - val_acc: 0.5340
Epoch 2/5
7982/7982 [==============================] - 1s 124us/step - loss: 1.9500 - acc: 0.5770 - val_loss: 1.6320 - val_acc: 0.6270
Epoch 3/5
7982/7982 [==============================] - 1s 121us/step - loss: 1.3855 - acc: 0.6988 - val_loss: 1.3518 - val_acc: 0.7070
Epoch 4/5
7982/7982 [==============================] - 1s 122us/step - loss: 1.0968 - acc: 0.7605 - val_loss: 1.1884 - val_acc: 0.7410
Epoch 5/5
7982/7982 [==============================] - 1s 119us/step - loss: 0.8775 - acc: 0.8104 - val_loss: 1.0759 - val_acc: 0.7680


In [8]:
model.evaluate(x_test, one_hot_test_labels)

2246/2246 [==============================] - 0s 130us/step


[1.1277016069033479, 0.748886910062333]

# Embedding

In [9]:
from keras.layers import Embedding
from keras import preprocessing

In [10]:
x_train = preprocessing.sequence.pad_sequences(train_data, maxlen=20) # 가장 빈번한 단어 최대 maxlen(20)개만
x_test = preprocessing.sequence.pad_sequences(test_data, maxlen=20)

In [11]:
x_train.shape

(8982, 20)

In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()

# 가능한 토큰의 개수(1,0000)와 임베딩 차원(8)로 설정 
model.add(Embedding(10000, 256, input_length=20))
# Embedding 층의 출력 크기는 (samples, maxlen, 8)가 됨

model.add(Flatten())  #(samples, maxlen * 8) 크기의 2D 텐서로 펼침


# 분류기를 추가
model.add(Dense(46, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()


####### 학습

history = model.fit(x_train, one_hot_train_labels,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 256)           2560000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 46)                235566    
Total params: 2,795,566
Trainable params: 2,795,566
Non-trainable params: 0
_________________________________________________________________
Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 10s 1ms/step - loss: 2.1898 - acc: 0.4703 - val_loss: 1.8181 - val_acc: 0.5326
Epoch 2/10
7185/7185 [==============================] - 9s 1ms/step - loss: 1.3338 - acc: 0.6668 - val_loss: 1.6133 - val_acc: 0.5888
Epoch 3/10
7185/7185 [==============================] - 9s 1ms/step - loss: 0.6

In [13]:
model.evaluate(x_test, one_hot_test_labels)

2246/2246 [==============================] - 0s 30us/step


[1.8437447456621423, 0.5886019590648284]